In [1]:
using LightGraphs
using DataFrames



"""
STABLE
inverse logit function
"""
invLogit(x) = 1./(1.+e.^-x)   

"""
STABLE
given graph and probability, adds a node which must have >0
connections by flipping biased coin for each existing node
"""
function addNode2(graph, p)
    add_vertex!(graph)
    x = nv(graph)
    degree = 0
    while degree ==0
        flips = rand(x-1)
        for i = 1:x-1
            if p[i]>flips[i]
                add_edge!(graph,i,x)
                degree +=1 
            end
        end
    end
    return graph
end


"""
STABLE
given graph, b vector, and a_0, adds a new node as specifiec by the model
"""
function addPrefNode(g,b,a_0 = -7)
    n = nv(g)
    L::SparseMatrixCSC{Int64,Int64} = laplacian_matrix(g)
    a::Array{Float64,1} = lufact(L) \ (b - mean(b))
    p::Array{Float64,1} = invLogit(a+a_0)
    addNode2(g,p)
    push!(b,0)
    return g
end


"""
STABLE
given graph and number of new edges desired, randomly adds edges between existing nodes
"""
function randEdgeGen(graph, newedges)
    for i in 1:newedges
        z = newedges
        x = collect(1:nv(graph))
        edge1 = rand(x)
        deleteat!(x, edge1)
        edge2 = rand(x)
        add_edge!(graph,edge1,edge2)
    end
    return graph
end
;

In [2]:
#outputs same as input

invLogit(x) = 1./(1.+e.^-x)   



#outputs same as input
soft(c,lambda) = sign(c).*max(abs(c)-lambda/2,0)

#


function gradient(a,a_0,u,L,rho,b,y)
    grad = -1.*(y-invLogit(a+a_0))+L*u + rho*L*(L*a-b)
    return grad
end;

function hessian(a,a_0,rho,L)
    hess = Diagonal(vec((invLogit(a+a_0).*(1-invLogit(a+a_0)))))+rho*L^2
    return hess
end;



function newton(y,a_0,L,rho,b,u)
    a= zeros(length(y))
    a_old = a
    iters = 0
    diff = 1.0
    while(diff >STOP_DIFF && iters< MAX_ITER )
        grad = gradient(a_old,a_0,u,L,rho,b,y)
        hess = hessian(a_old,a_0, rho,L)
        a = a_old - inv(hess)*grad
        diff = norm(a-a_old)
        a_old = a
        iters = iters+1
    end
    if(iters == MAX_ITER)
        print("max iter reached")
    end
    return a
end
;



function ADMM(A,L, rho, lambda, a_0)
    t_0 = length(A[1])
    t = length(A[size(A)[1]])
	new = size(A)[1] 
    a = Array{Float64,1}[]
    u = Array{Float64,1}[]
    for i in t_0:t
        push!(a,zeros(i)+0.0)
        push!(u,zeros(i)+0.0)
    end
	b = zeros(t_0)
	iters = 0
	diff = 1.0
	b_old = b
	while(diff >STOP_DIFF && iters< MAX_ITER )
		for i in 1:new
            a[i] = newton(A[i],a_0,L[i],rho,vcat(b,zeros(i-1)),u[i])
        end
        c = zeros(t)
        for i in 1:new 
        	c[1:(t_0 +i-1)] = c[1:(t_0 +i-1)]+ (u[i]+rho*(L[i]*a[i]))/(rho*new)
        end
        b = soft(c[1:t_0],2*lambda/rho)
        #u update
        for i in 1:new
            u[i] = u[i]+ rho*(L[i]*a[i]-vcat(b,zeros(i-1)))
        end
        diff  = norm(b-b_old)
        b_old = b
        println(diff)
    end
    return(b)
end;

# Using arXiv data

In [3]:
const MAX_ITER = 1000
const STOP_DIFF = 0.001;

rho = 1
lambda = 0.0005

using LightGraphs
using DataFrames


data = readtable("training.csv")
;#change directory

0.0005

In [24]:
t = maximum([maximum(data[:,4]),maximum(data[:,5])])
g = BinaryTree(0)
for i in 1:t
	add_vertex!(g)
end
for i in 1:size(data,1)
	add_edge!(g,data[i,4],data[i,5])
end
At = adjacency_matrix(g)


A = Array{Int64,1}[]
L =  SparseMatrixCSC{Int64,Int64}[]
Adj =  SparseMatrixCSC{Int64,Int64}[]

new = 25
for i in reverse(1:new)
    n = size(At)[1]
    push!(A,full(At[n-i+1,1:n-i]))
    push!(Adj, At[1:n-i,1:n-i])
end

for adj in Adj
    push!(L,spdiagm(vec(sum(Adj[1],1))).-Adj[1])
end
    

In [21]:
ADMM(A,L,rho,lambda,-5)

LoadError: Base.LinAlg.SingularException(1055)

# Simulation Data

# original, not enough variation

In [22]:
levels = 10     #number of levels in binary tree
g = BinaryTree(levels)
n = nv(g)
b = (rand(n) .< 8 / n)*5. 
genb = copy(b)  # save for later
g = randEdgeGen(g,10000)
A = Array{Int64,1}[]
L =  SparseMatrixCSC{Int64,Int64}[]
numnewnodes = 5
a_0 = -4
# creates matrix A and L where A[i] is the connections for ith node and L[i] is the laplacian of the i-1st time step 
for i in 1:numnewnodes 
    push!(L, laplacian_matrix(g))
    g = addPrefNode(g,b, a_0)
    connects = zeros(2^levels-2+i)  #-1 for -1 1 coding
    connects[neighbors(g,nv(g))] = 1
    push!(A,connects)
end

In [23]:
ADMM(A,L,rho,lambda,-4)

0.08568649948364195
0.08530497423582022
0.08509737609258956
0.08502236046596003
0.08501661849741912
0.08501066326179078
0.08500510567071862
0.08499962638676105
0.0849941938804803
0.08498876472821898
0.08498333070585413
0.08497788846984874
0.08497245016751975
0.08496700756978444
0.08496156515137868
0.08495611959987627
0.08495067161260761
0.08494522323487837


LoadError: InterruptException:

# more variation needs work


In [ ]:
const MAX_ITER = 1000
const STOP_DIFF = 0.001;

rho = 1
lambda = 0.0005

In [1]:
using Distributions

In [9]:
levels = 10     #number of levels in binary tree
g = BinaryTree(levels)
n = nv(g)

#dist = Gamma(2,2)
#b = 50.*rand(dist,n)
b = vec(readdlm("bvec.dat"))

genb = copy(b)  # save for later
g = randEdgeGen(g,10000)
A = Array{Int64,1}[]
L =  SparseMatrixCSC{Int64,Int64}[]
a_0 = 62
numnewnodes = 5


5

In [10]:
# creates matrix A and L where A[i] is the connections for ith node and L[i] is the laplacian of the i-1st time step 
for i in 1:numnewnodes 
    push!(L, laplacian_matrix(g))
    g = addPrefNode(g,b,a_0 )
    connects = zeros(2^levels-2+i)  #-1 for -1 1 coding
    connects[neighbors(g,nv(g))] = 1
    push!(A,connects)
end